In [9]:
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib.pyplot import rc_context
from tqdm import tqdm
#from scipy.optimize import curve_fit

#import scvi
import pickle
from adjustText import adjust_text

import itertools
#from numba import jit

import gseapy as gp
import random

from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import dendrogram, linkage

In [10]:
with open('../data/files/5_TMS_HUBS_DICTIONARY_FILTERED.pickle', 'rb') as handle:
    hubs = pickle.load(handle)
    
hub_metadata = pd.read_pickle('../data/files/5_TMS_HUBS_METADATA_FILTERED.pickle')


with open('../data/files/5_TMS_SIGS_DICTIONARY_FILTERED.pickle', 'rb') as handle:
    signatures = pickle.load(handle)
    
sig_metadata = pd.read_pickle('../data/files/5_TMS_SIGS_METADATA_FILTERED.pickle')

#with open('../data/files/5_TMS_HUBS_NxN_corrs.pickle', 'rb') as handle:
#    hub_Ms = pickle.load(handle)

In [3]:
adata = sc.read_h5ad('../data/tms/tms-scVI-raw-data_BDATA.h5ad')

## Gene-based

In [4]:
signatures

{('Bladder', 'bladder cell'): [('Sfrp5', 23),
  ('Agtr1a', 30),
  ('Rasgef1c', 17),
  ('Rxrg', 21),
  ('Plce1', 30),
  ('Kcna2', 23),
  ('Slc35f1', 20),
  ('Ppp1r14c', 7),
  ('Cadm4', 27),
  ('Matn4', 13),
  ('Gfra3', 20),
  ('Lgi1', 18),
  ('Tmprss5', 17),
  ('Mpz', 21),
  ('Ngfr', 9),
  ('Cadm2', 32),
  ('Fam89a', 20),
  ('Nat8l', 16),
  ('Klk8', 19),
  ('Kcna1', 23),
  ('Syt7', 13),
  ('Map3k9', 5),
  ('Msln', 31),
  ('Slpi', 27),
  ('Dpp4', 44),
  ('Pcnxl2', 27),
  ('Pkhd1l1', 33),
  ('Slurp1', 29),
  ('Podxl', 33),
  ('Gpr64', 29),
  ('Col8a2', 28),
  ('Prr15l', 33),
  ('Cdon', 27),
  ('Nrxn2', 15),
  ('Prlr', 26),
  ('Ablim3', 29),
  ('Hoxc6', 23),
  ('Bex1', 8),
  ('Sgcd', 18),
  ('Slc4a8', 25),
  ('Myl7', 32),
  ('Rspo1', 30),
  ('Wt1', 38),
  ('Gpm6a', 36),
  ('Mira', 4),
  ('Cdh3', 37),
  ('Clic3', 26),
  ('Hoxd10', 22),
  ('Fgf9', 18),
  ('Hoxd13', 10),
  ('Espn', 26),
  ('Hoxc4', 14),
  ('Smpd3', 36),
  ('Me3', 10),
  ('Kcnj8', 20),
  ('Plscr2', 19),
  ('Rnf165', 3),
  ('Cc

In [5]:
out = []
#for a,b in itertools.combinations(list(signatures), 2):
for a,b in itertools.product(list(signatures), list(signatures)):   
    al = [x[0] for x in signatures[a]]
    bl = [x[0] for x in signatures[b]]
    
    p = stats.hypergeom.sf(len([x for x in al if x in bl]) - 1, len(adata.var_names), len(al), len(bl))
    
    
    if al == bl:
        p = 0
        
    if p == 0 or p == 1:
        if p == 0:
            v = 1/300
        else:
            v = 1
        
    
    else:
        v = 1/-np.log10(p)
    
    out.append([
        a[0] + ' ' +a[1],
        b[0] + ' ' +b[1],
        p,
        v
        
    ])

In [6]:
dists = pd.DataFrame(out, columns = ['a', 'b', 'p', 'v'])


#dists['dist'] = -np.log10(dists.p)


dists = dists.pivot(index = 'a', columns = 'b', values = 'v')

dists

b,Bladder bladder cell,Bladder endothelial cell,Brain endothelial cell,Brain oligodendrocyte,Diaphragm mesenchymal stem cell,Diaphragm skeletal muscle satellite cell,Fat endothelial cell,Fat mesenchymal stem cell,HSC hematopoietic stem cell,Heart_and_Aorta endocardial cell,...,Myeloid monocyte,Pancreas pancreatic A cell,Pancreas pancreatic ductal cell,Skin epidermal cell,Thymus thymocyte,Tongue basal cell,Tongue keratinocyte,Trachea basal cell,Trachea fibroblast,Trachea macrophage
a,,,,,,,,,,,,,,,,,,,,,
Bladder bladder cell,0.003333,0.161183,1.000000,1.000000,0.242827,0.198472,0.167516,0.107941,2.699082,0.132000,...,1.467703,7.881158,1.000000,1.156008,0.646554,0.678945,0.363798,0.195080,0.126767,0.116721
Bladder endothelial cell,0.161183,0.003333,1.000000,0.948613,0.789835,0.770200,0.466751,0.664637,1.170200,0.090680,...,1.000000,2.986607,1.000000,1.000000,0.612376,1.778474,2.809479,1.423207,0.406678,0.292520
Brain endothelial cell,1.000000,1.000000,0.003333,1.000000,0.126772,0.219638,0.405042,1.000000,0.368586,0.150202,...,0.172240,0.952723,1.000000,1.000000,0.368476,0.641386,0.514689,0.350947,0.118018,1.578497
Brain oligodendrocyte,1.000000,0.948613,1.000000,0.003333,0.379566,0.366674,2.511380,0.322959,0.863716,0.389139,...,1.000000,2.112748,1.000000,1.326276,0.419495,1.388705,1.000000,0.911934,1.181242,0.347822
Diaphragm mesenchymal stem cell,0.242827,0.789835,0.126772,0.379566,0.003333,0.005587,0.230136,0.188519,0.036300,0.011507,...,0.604353,0.017686,8.082570,2.197078,0.033505,0.084015,0.062955,0.015880,0.007901,0.074052
Diaphragm skeletal muscle satellite cell,0.198472,0.770200,0.219638,0.366674,0.005587,0.003333,0.212245,0.266097,0.029038,0.011607,...,0.566578,0.016776,4.264427,0.497244,0.069172,0.081331,0.062631,0.014186,0.009675,0.048897
Fat endothelial cell,0.167516,0.466751,0.405042,2.511380,0.230136,0.212245,0.003333,0.071792,0.335926,0.075962,...,1.059354,0.595752,0.454223,1.507422,0.580177,0.995052,0.545487,0.742862,0.207368,0.187604
Fat mesenchymal stem cell,0.107941,0.664637,1.000000,0.322959,0.188519,0.266097,0.071792,0.003333,0.723998,0.109012,...,1.382283,2.168149,0.260516,1.000000,0.735636,3.050893,1.030113,0.933174,0.174103,0.913119
HSC hematopoietic stem cell,2.699082,1.170200,0.368586,0.863716,0.036300,0.029038,0.335926,0.723998,0.003333,0.059051,...,0.060166,0.025061,0.625775,1.251706,0.043327,0.321795,0.212191,0.035672,0.035518,0.052428


In [7]:
condensed = []
for x, row in enumerate(dists.iterrows()):
    vals = row[1].tolist()
    condensed += vals[x+1:]

In [11]:
linkage_data = linkage(condensed)

### Sup Fig 4

In [13]:
#hierarchy.set_link_color_palette(['k', 'y'])

with rc_context({'lines.linewidth': 5}):


    plt.figure(figsize = (6,12))

    ax = dendrogram(linkage_data, orientation = 'left', labels = dists.index)
    

    plt.xticks(size = 10,  weight = 'bold')
    plt.yticks(size = 10, weight = 'bold')


    plt.savefig('figures/S4.png', dpi = 600, bbox_inches = 'tight')
    plt.savefig('figures/S4.svg', bbox_inches = 'tight')